In [1]:
import json
import ROOT
from ROOT import RDataFrame, TCanvas, THStack
ROOT.EnableImplicitMT()
import numpy as np
%jsroot on

Welcome to JupyROOT 6.26/04


In [2]:
#redeclaration is annoying, becouse you can not modify
@ROOT.Numba.Declare(["unsigned int", "ROOT::VecOps::RVec<float>&"], "ROOT::VecOps::RVec<double>")
def btag_weight_variation(i_jet, jet_pt):
    return 1 + np.array((.075,-.075))*(jet_pt[i_jet]/50)

/cvmfs/sft.cern.ch/lcg/views/LCG_102/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_102/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/cvmfs/sft.cern.ch/lcg/views/LCG_102/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:499: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/sft.cern.ch/lcg/views/LCG_102/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  r

In [3]:
@ROOT.Numba.Declare([], "double")
def pt_nominal():
    return 1.

In [4]:
@ROOT.Numba.Declare([], "double")
def pt_scale_up():
    return 1.03

In [5]:
@ROOT.Numba.Declare([],"ROOT::VecOps::RVec<double>")
def flat_variation():
    # 0.1% weight variations
    return (1.0 + np.array([0.025, -0.025], dtype=np.float32))

In [6]:
# @ROOT.Numba.Declare(["ROOT::VecOps::RVec<double>&"], "ROOT::VecOps::RVec<double>")
# def pt_res_up(jet_pt):
#     return np.random.normal(1, 0.05, len(jet_pt)) 

In [7]:
# ROOT.gSystem.SetIncludePath("-I$ROOTSYS/include")
ROOT.gSystem.CompileMacro("helper.cpp", "kO")

1

In [8]:
from urllib.request import urlretrieve
import os

class TtbarAnalysis(dict):

    def __init__(self, n_files_max_per_sample = 1, num_bins=25, bin_low = 50, bin_high = 550, download_input_data=False, use_local_data=False):
        
        self.download_input_data = download_input_data
        self.use_local_data = use_local_data
        self._nevts_total = {}
        self.n_files_max_per_sample = n_files_max_per_sample
        self.input_data = self._construct_fileset()
        self.num_bins = num_bins
        self.bin_low = bin_low
        self.bin_high = bin_high
        self.xsec_info = {
            
            "ttbar": 396.87 + 332.97, # nonallhad + allhad, keep same x-sec for all
            "single_top_s_chan": 2.0268 + 1.2676,
            "single_top_t_chan": (36.993 + 22.175)/0.252,  # scale from lepton filter to inclusive
            "single_top_tW": 37.936 + 37.906,
            "wjets": 61457 * 0.252,  # e/mu+nu final states
            "data": None
        }
    
        

    def _construct_fileset(self):
        n_files_max_per_sample = self.n_files_max_per_sample
        with open ('ntuples.json') as f:
            file_info = json.load(f)
        fileset = {}
        for process in file_info.keys():
            if process == "data":
                continue  # skip data
            fileset[process] = {}
            self[process] = {}
            self._nevts_total[process] = {}
            for variation in file_info[process].keys():
#                 if variation != 'nominal': continue      
                file_list = file_info[process][variation]["files"]
                if n_files_max_per_sample != -1:
                    file_list = file_list[:n_files_max_per_sample]  # use partial set of samples
                file_paths = [f["path"] for f in file_list]
                nevts_total = sum([f["nevts"] for f in file_list])
                self._nevts_total[process].update({variation:nevts_total})
                fileset[process].update({variation: file_paths})
                if (self.download_input_data):
                    dir_name = f"input/{process}_{variation}"
                    os.makedirs(dir_name, exist_ok=True)
                    for i in range(len(file_paths)):
                        path = file_paths[i]
                        file = f"{dir_name}/{i}.root"
                        if not os.path.exists(file):
                            urlretrieve(path, file)   
                            print(f"directory {dir_name} has been created")
                        else:
                            print(f"{file} already exists")
                self[process][variation] = {}
        return fileset

    def fill(self, process, variation):

        input_data = f"root://eosuser.cern.ch//eos/user/a/afalko/analysis-grand-challenge/analyses/cms-open-data-ttbar/input/{process}_{variation}/*.root" if self.use_local_data else self.input_data[process][variation]               
        d = RDataFrame('events', input_data)

        x_sec = self.xsec_info[process]
        nevts_total = self._nevts_total[process][variation]
        lumi = 3378 # /pb
        xsec_weight = x_sec * lumi / nevts_total

        pt_variations = ["pt_nominal", "pt_scale_up", "pt_res_up"] if variation == "nominal" else ["pt_nominal"]
        
        for pt_var in pt_variations:
            jet_pt_modified = f"jet_pt*Numba::{pt_var}()" if "res" not in pt_var else f"jet_pt*pt_res_up(jet_pt)"
            d_mod = d.Redefine('jet_pt', jet_pt_modified)
            d_mod = d_mod.Define('electron_pt_mask', 'electron_pt>25').Define('muon_pt_mask', 'muon_pt>25').Define('jet_pt_mask', 'jet_pt>25')\
                 .Filter('Sum(electron_pt_mask) + Sum(muon_pt_mask) == 1')\
                 .Filter('Sum(jet_pt_mask) >= 4')\
                 .Filter('Sum(jet_btag[jet_pt_mask]>=0.5)>=1')\
                 .Define('weights', str(xsec_weight)) 


    #         #it gives number 1
    #         print(d.GetNRuns())
            measured = {"4j1b": "HT", "4j2b": 'trijet_mass'}
            for region in ["4j1b","4j2b"]:
                meas = measured[region]
                if region == "4j1b":

                    fork = d_mod.Filter('Sum(jet_btag[jet_pt_mask]>=0.5)==1').Define(meas, 'Sum(jet_pt[jet_pt_mask])')      

                elif region == "4j2b":

                    fork = d_mod.Filter('Sum(jet_btag[jet_pt_mask]>=0.5)>1').Define("jet_p4", 
                        "ROOT::VecOps::Construct<ROOT::Math::PxPyPzMVector>(jet_px[jet_pt_mask], jet_py[jet_pt_mask], jet_pz[jet_pt_mask], jet_mass[jet_pt_mask])"
                    )

                    fork = fork.Define('trijet', 
                        'ROOT::VecOps::Combinations(jet_pt[jet_pt_mask],3)'
                    ).Define('ntrijet', 'trijet[0].size()')

                    fork = fork.Define('trijet_p4', 
                                          'ROOT::VecOps::RVec<ROOT::Math::PxPyPzMVector> trijet_p4(ntrijet);'              +\
                                          'for (int i = 0; i < ntrijet; ++i) {'                                            +\
                                              'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'       +\
                                              'trijet_p4[i] = jet_p4[j1] + jet_p4[j2] + jet_p4[j3];'                       +\
                                          '}'                                                                              +\
                                          'return trijet_p4;'                                                                                                                          
                                         )

                    #TODO  implement references
                    fork = fork.Define('trijet_pt', 
                            'return ROOT::VecOps::Map(trijet_p4, [](ROOT::Math::PxPyPzMVector v) { return v.Pt(); })'
                                                )

                    fork = fork.Define('trijet_btag', 
                                                      'ROOT::VecOps::RVec<bool> btag(ntrijet);'                                   +\
                                                      'for (int i = 0; i < ntrijet; ++i) {'                                       +\
                                                       'int j1 = trijet[0][i]; int j2 = trijet[1][i]; int j3 = trijet[2][i];'     +\
                                                       'btag[i]=std::max({jet_btag[j1], jet_btag[j2], jet_btag[j3]})>0.5;'        +\
                                                      '}'                                                                         +\
                                                      'return btag;'
                                                )

                    fork=fork.Define(meas,
                                                      'double mass;'+\
                                                      'double Pt = 0;'+\
                                                      'double indx = 0;'+\
                                                      'for (int i = 0; i < ntrijet; ++i) {'               +\
                                                      '    if ((Pt < trijet_pt[i]) && (trijet_btag[i])) {'+\
                                                      '        Pt = trijet_pt[i];'+\
                                                      '        indx=i;'+\
                                                      '    }'                                            +\
                                                      '}'                                                +\
                                                      'mass = trijet_p4[indx].M();'             +\
                                                      'return mass;'
                                                     )

                if "nominal" in pt_var:   
                    res = fork.Histo1D((f'{process}_{variation}_{region}', process, self.num_bins, self.bin_low, self.bin_high), meas, 'weights')
                    self[process][variation][region] = res
                    self.hist.append(res)
                    print(f'histogram {region}_{process}_{variation} has been created')
                    
                    if variation == "nominal":
                        for i_var, weight_name in enumerate([f"btag_var_{i}" for i in range(4)]):
                            for i_dir, direction in enumerate(["up", "down"]):

                                btag_weight_variation=f"{weight_name}_{direction}"
                                var_weights = f'weights_{btag_weight_variation}_{region}'
                                fork = fork.Define(var_weights, 
                                           f"weights*Numba::btag_weight_variation({i_var}, jet_pt[jet_pt_mask])[{i_dir}]"
                                        )

                                res = fork.Histo1D(
                                                    (f'{process}_{btag_weight_variation}_{region}', btag_weight_variation, self.num_bins, self.bin_low, self.bin_high),
                                                    meas,
                                                    var_weights
                                )


                                if btag_weight_variation not in self[process]:
                                    self[process][btag_weight_variation] = {}

                                self[process][btag_weight_variation][region] = res                   

                                self.hist.append(res)

                        if process == 'wjets':
                            for i_dir, direction in enumerate(["up", "down"]):
                                weight_var = f"scale_var_{direction}"
                                res = fork.Define(weight_var, f"weights*Numba::flat_variation()[{i_dir}]")\
                                            .Histo1D(
                                                    (f"{region}_{process}_{weight_var}", weight_var, self.num_bins, self.bin_low, self.bin_high),
                                                    meas,
                                                    weight_var
                                                    )
                                if weight_var not in self[process]:
                                    self[process][weight_var] = {}
                                self[process][weight_var][region] = res
                                self.hist.append(res)
                                # print(f'histogram {region}_{process}_{weight_var} has been created')
                else:
                    res = fork.Histo1D((f'{process}_{pt_var}_{region}', process, self.num_bins, self.bin_low, self.bin_high), meas, 'weights')
                    if pt_var not in self[process]:
                         self[process][pt_var] = {}
                    self[process][pt_var][region] = res
                    self.hist.append(res)
#                     print(f'histogram {region}_{process}_{pt_var} has been created')                    



            

    def Fill(self):
        self.hist = []
        for process in self:
            
            for variation in self.input_data[process]:
                self.fill(process=process, variation=variation)

    def Accumulate(self):
        ROOT.RDF.RunGraphs(self.hist)            
        
    def GetProcStack(self, region, variation='nominal'):
        return [self[process][variation][region] for process in self]
    
    def GetVarStack(self, region, process="ttbar"):
        return [self[process][variation][region] for variation in self[process]]

    

In [9]:
analysisManager = TtbarAnalysis(download_input_data=True, use_local_data=True)
analysisManager.input_data

input/ttbar_nominal/0.root already exists
input/ttbar_scaledown/0.root already exists
input/ttbar_scaleup/0.root already exists
input/ttbar_ME_var/0.root already exists
input/ttbar_PS_var/0.root already exists
input/single_top_s_chan_nominal/0.root already exists
input/single_top_t_chan_nominal/0.root already exists
input/single_top_tW_nominal/0.root already exists
input/wjets_nominal/0.root already exists


{'ttbar': {'nominal': ['https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-pythia8/MINIAODSIM//PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/00000/00DF0A73-17C2-E511-B086-E41D2D08DE30.root'],
  'scaledown': ['https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-scaledown-pythia8/MINIAODSIM/PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/00000/00D45978-82D1-E511-9406-00237DF27440.root'],
  'scaleup': ['https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-powheg-scaleup-pythia8/MINIAODSIM/PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext3-v1/10000/002A13F7-B8CB-E511-A112-003048CB7B30.root'],
  'ME_var': ['https://xrootd-local.unl.edu:1094//store/user/AGC/datasets/RunIIFall15MiniAODv2/TT_TuneCUETP8M1_13TeV-amcatnlo-pythia8/MINIAODSIM/PU25nsData2015v1_76X_mcRun2_asymptotic_v12_ext1-v1/60000/00F8750A-D4CB-E511-A1D1-00215E2EAD

In [10]:
import time
t0 = time.time()
analysisManager.Fill()
t1 = time.time()
print(f"\npreprocessing took {round(t1 - t0,2)} seconds")
analysisManager.Accumulate()
t2 = time.time()
print(f"processing took {round(t2 - t1,2)} seconds")
print(f"execution took {round(t2 - t0,2)} seconds")

histogram 4j1b_ttbar_nominal has been created
histogram 4j2b_ttbar_nominal has been created
histogram 4j1b_ttbar_scaledown has been created
histogram 4j2b_ttbar_scaledown has been created
histogram 4j1b_ttbar_scaleup has been created
histogram 4j2b_ttbar_scaleup has been created
histogram 4j1b_ttbar_ME_var has been created
histogram 4j2b_ttbar_ME_var has been created
histogram 4j1b_ttbar_PS_var has been created
histogram 4j2b_ttbar_PS_var has been created
histogram 4j1b_single_top_s_chan_nominal has been created
histogram 4j2b_single_top_s_chan_nominal has been created
histogram 4j1b_single_top_t_chan_nominal has been created
histogram 4j2b_single_top_t_chan_nominal has been created
histogram 4j1b_single_top_tW_nominal has been created
histogram 4j2b_single_top_tW_nominal has been created
histogram 4j1b_wjets_nominal has been created
histogram 4j2b_wjets_nominal has been created

preprocessing took 2.69 seconds
processing took 16.76 seconds
execution took 19.44 seconds


In [11]:
output = ROOT.TFile.Open('rdf.root', 'RECREATE')
for process in analysisManager:
    for variation in analysisManager[process]:
        for region in analysisManager[process][variation]:
            hist_name = f"{region}_{process}_{variation}" if variation != 'nominal' else f"{region}_{process}"
            hist_ptr = analysisManager[process][variation][region].GetPtr()
            hist_sliced = ROOT.Slice(hist_ptr, 120, 550)
            hist_binned = hist_sliced.Rebin(2, hist_ptr.GetTitle())
            output.WriteObject(hist_binned, hist_name)
output.Close()

Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_ttbar (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_btag_var_0_up (Potential memory leak).
Warning in <TFile::Append>: Replacing existing TH1: h_sliced_btag_var_0_

In [12]:
rdf = ROOT.TFile.Open('rdf.root')
coffea = ROOT.TFile.Open('histograms.root')
rdf.ls()
print(rdf.GetNkeys())

122
TFile**		rdf.root	
 TFile*		rdf.root	
  KEY: TH1D	4j1b_ttbar;1	ttbar
  KEY: TH1D	4j2b_ttbar;1	ttbar
  KEY: TH1D	4j1b_ttbar_scaledown;1	ttbar
  KEY: TH1D	4j2b_ttbar_scaledown;1	ttbar
  KEY: TH1D	4j1b_ttbar_scaleup;1	ttbar
  KEY: TH1D	4j2b_ttbar_scaleup;1	ttbar
  KEY: TH1D	4j1b_ttbar_ME_var;1	ttbar
  KEY: TH1D	4j2b_ttbar_ME_var;1	ttbar
  KEY: TH1D	4j1b_ttbar_PS_var;1	ttbar
  KEY: TH1D	4j2b_ttbar_PS_var;1	ttbar
  KEY: TH1D	4j1b_ttbar_btag_var_0_up;1	btag_var_0_up
  KEY: TH1D	4j2b_ttbar_btag_var_0_up;1	btag_var_0_up
  KEY: TH1D	4j1b_ttbar_btag_var_0_down;1	btag_var_0_down
  KEY: TH1D	4j2b_ttbar_btag_var_0_down;1	btag_var_0_down
  KEY: TH1D	4j1b_ttbar_btag_var_1_up;1	btag_var_1_up
  KEY: TH1D	4j2b_ttbar_btag_var_1_up;1	btag_var_1_up
  KEY: TH1D	4j1b_ttbar_btag_var_1_down;1	btag_var_1_down
  KEY: TH1D	4j2b_ttbar_btag_var_1_down;1	btag_var_1_down
  KEY: TH1D	4j1b_ttbar_btag_var_2_up;1	btag_var_2_up
  KEY: TH1D	4j2b_ttbar_btag_var_2_up;1	btag_var_2_up
  KEY: TH1D	4j1b_ttbar_btag_var_2_down

In [13]:
coffea.ls()
print(coffea.GetNkeys())

124
TFile**		histograms.root	
 TFile*		histograms.root	
  KEY: TH1D	4j1b_pseudodata;1	
  KEY: TH1D	4j1b_ttbar;1	
  KEY: TH1D	4j1b_ttbar_btag_var_0_up;1	
  KEY: TH1D	4j1b_ttbar_btag_var_0_down;1	
  KEY: TH1D	4j1b_ttbar_btag_var_1_up;1	
  KEY: TH1D	4j1b_ttbar_btag_var_1_down;1	
  KEY: TH1D	4j1b_ttbar_btag_var_2_up;1	
  KEY: TH1D	4j1b_ttbar_btag_var_2_down;1	
  KEY: TH1D	4j1b_ttbar_btag_var_3_up;1	
  KEY: TH1D	4j1b_ttbar_btag_var_3_down;1	
  KEY: TH1D	4j1b_ttbar_pt_scale_up;1	
  KEY: TH1D	4j1b_ttbar_pt_res_up;1	
  KEY: TH1D	4j1b_single_top_s_chan;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_0_up;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_0_down;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_1_up;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_1_down;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_2_up;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_2_down;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_3_up;1	
  KEY: TH1D	4j1b_single_top_s_chan_btag_var_3_down;1	
  KEY: TH1D	4j1b_single

In [14]:
def get_values(h):
    return np.array([h.GetBinContent(i+1) for i in range (h.GetNbinsX())])

def match_histos (rdf_hist, coffea_hist, precision):
    rdf_values = get_values(rdf_hist)
    coffea_values = get_values(coffea_hist)
    rdf_values = np.round(rdf_values, precision)
    coffea_values = np.round(coffea_values, precision)
    mask = rdf_values == coffea_values
    return not (False in mask)

def deviations(rdf_hist, coffea_hist):
    rdf_values = get_values(rdf_hist)
    coffea_values = get_values(coffea_hist)
    deviations = np.zeros(len(rdf_values))
    for i in range(len(deviations)):
        rdf_value = rdf_values[i]
        coffea_value = coffea_values[i]
        deviations[i] = 0 if round(coffea_value, 5) == round(rdf_value, 5) == 0 else 100*abs(rdf_value-coffea_value)/coffea_value
#         deviations[i] = 100*abs(rdf_value-coffea_value)/coffea_value

    return deviations


def get_deviations ():
    mismatched = []
    for process in analysisManager:
        for variation in analysisManager[process]:
            for region in analysisManager[process][variation]:
                hist_name = f"{region}_{process}_{variation}" if variation != 'nominal' else f"{region}_{process}"
                rdf_hist = rdf.Get(hist_name)
                coffea_hist = coffea.Get(hist_name)
                if (rdf_hist and coffea_hist):
                    deviation = np.max(deviations(rdf_hist, coffea_hist))
#                     if deviation > 10:
                    if deviation > 0.0001 and "res_up" not in hist_name:
                        print(f"{hist_name} maximum deviation, %: {deviation}")
                        mismatched.append(hist_name)
                else:
                    raise ValueError('rdf_hist and coffea_hist is Zombie')
    return mismatched
# def compare_histos(precision):
#     for process in analysisManager:
#         for variation in analysisManager[process]:
#             for region in analysisManager[process][variation]:
# #                 if "pt" in variation and "2" in region:
# #                     continue
#                 hist_name = f"{region}_{process}_{variation}" if variation != 'nominal' else f"{region}_{process}"
#                 rdf_hist = rdf.Get(hist_name)
#                 coffea_hist = coffea.Get(hist_name)
#     #             print(hist_name,'is processing')
#                 if (rdf_hist and coffea_hist):
#                     deviation = np.max(deviations(rdf_hist, coffea_hist))
#                     if deviation >= 0.1:
#                         print(f"{hist_name} maximum deviation, %: {deviation}")
# #                     if not match_histos(rdf_hist, coffea_hist, precision):
# #                         print('mismatch', hist_name)
#                 else:
#                     raise ValueError('rdf_hist and coffea_hist is Zombie')



In [15]:
mismatched = get_deviations()

In [16]:
hist = '4j2b_wjets_pt_res_up'
c0 = TCanvas()
h1 = rdf.Get(hist); h2 = coffea.Get(hist)
c0.Divide(2,1)
c0.cd(1)
h1.Draw('hist pfc plc')
c0.cd(2)
h2.Draw('hist pfc plc')
c0.Draw()

In [17]:
c = TCanvas()
c.Print('deviation_larger_10.pdf[')
for hist in mismatched :
    h1 = rdf.Get(hist); h2 = coffea.Get(hist)
    c.Clear()
    c.Divide(2,1)
    c.cd(1)
    h1.Draw('hist pfc plc')
    c.cd(2)
    h2.Draw('hist pfc plc')
    c.Print('deviation_larger_10.pdf')
c.Print('deviation_larger_10.pdf]')

Info in <TCanvas::Print>: pdf file deviation_larger_10.pdf has been created
Info in <TCanvas::Print>: pdf file deviation_larger_10.pdf has been closed


In [18]:
c = TCanvas('c', 'c', 600, 400) 
hlist = analysisManager.GetProcStack(region='4j1b')
hs = THStack('j4b1', '>=4 jets, 1 b-tag (RDF); H_{T} [GeV]')
for h in hlist:
    ptr = h.Rebin(2, h.GetTitle())
    hs.Add(ptr)
hs.Draw('hist pfc plc')
c.Draw()
x = hs.GetXaxis()
x.SetRangeUser(120, 500)
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.


In [19]:
c = TCanvas('c', 'c', 600, 400) 
hlist = analysisManager.GetProcStack(region='4j2b')
hs = THStack('j4b1', '>=4 jets, 1 b-tag (RDF); H_{T} [GeV]')
for h in hlist:
    ptr = h.Rebin(2, h.GetTitle())
    hs.Add(ptr)
hs.Draw('hist pfc plc')
c.Draw()
x = hs.GetXaxis()
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.


In [20]:
freshstack = analysisManager.GetVarStack(region='4j1b')
hs = THStack('j4b1btag', 'b-tagging variations (RDF); H_{T} [GeV]')
for h in freshstack:
    ptr = h.Rebin(2, h.GetTitle())
    ptr.SetFillColor(0)
    ptr.SetLineWidth(1)
    hs.Add(ptr)
hs.Draw('hist nostack')
c.Draw()
x = hs.GetXaxis()
x.SetRangeUser(120, 500)
x.SetTitleOffset(1.5)
x.CenterTitle()
c.BuildLegend(0.65, 0.7, 0.9, 0.9)

Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nbins=25.
Warning in <TH1D::Rebin>: ngroup=2 is not an exact divider of nb